## 1. Introduction

### a. Project's goal

### b. Project's constraints

## 2. Computer Vision

### a. Scene initialization

### b. Detection of obstacles, Thymio and goal point

## 3. Global Navigation

### a. Path planning

### b. Robot's actuation

## 4 Local Navigation

### a. Obstacle avoidance (Artificial Neural Network)

### b. Completion of the obstacle avoidance

## 5. Position Estimation: Kalman Filter

### a. Empirical tests

### b. Kalman filtering theory

## 6. Video Demonstrations

### a. Global example

### b. Specific cases

#### i. Obstacle avoidance

#### ii. Kidnapping

#### iii. Change in goal's position

#### iv. Hidden Camera